In [1]:
using XLSX, DataFrames, TimeSeries
using StatsBase
using Plots, StatsPlots
using LinearAlgebra
using GLM
using FredData
include("utils.jl")

print_percentiles (generic function with 1 method)

In [2]:
f = Fred("b258d550a51a3c02c25400cc4267b3a8")

FRED API Connection
	url: https://api.stlouisfed.org/fred/
	key: b258d550a51a3c02c25400cc4267b3a8


In [165]:
Date(1971,10,1):Quarter(1):Date(2024,6,1) |> collect

211-element Vector{Date}:
 1971-10-01
 1972-01-01
 1972-04-01
 1972-07-01
 1972-10-01
 1973-01-01
 1973-04-01
 1973-07-01
 1973-10-01
 1974-01-01
 ⋮
 2022-04-01
 2022-07-01
 2022-10-01
 2023-01-01
 2023-04-01
 2023-07-01
 2023-10-01
 2024-01-01
 2024-04-01

In [169]:
get_data(f, string("CP0000EZ19M086NEST");observation_start="1971-10-01", frequency="q").data[:,[:date, :value]]

Row,date,value
,Date,Float64
1,1996-01-01,70.71
2,1996-04-01,71.26
3,1996-07-01,71.34
4,1996-10-01,71.59
5,1997-01-01,72.09
6,1997-04-01,72.32
7,1997-07-01,72.56
8,1997-10-01,72.79
9,1998-01-01,72.97


In [170]:
macro_data = TimeArray((date = Date(1971,10,1):Quarter(1):Date(2024,6,1), empty = Array{Float64}(undef,211)); timestamp = :date)
factors = [:NGDPRSAXDCPLQ, :POLCPIALLQINMEI, :IRLTLT01PLM156N, :IR3TIB01PLQ156N, :NGDPRSAXDCUSQ, :USACPIALLMINMEI, :IRLTLT01USQ156N, :IR3TIB01USQ156N, :CLVMEURSCAB1GQEA19, :CP0000EZ19M086NEST, :IRLTLT01EZM156N, :IR3TIB01EZQ156N]
for factor in factors
    temp = get_data(f, string(factor);observation_start="1971-10-01", frequency="q").data[:,[:date, :value]]
    temp = TimeArray(temp, timestamp = :date)
    TimeSeries.rename!(temp,[factor])
    macro_data = merge(macro_data, temp; method = :left)
end
macro_data


┌ Warning: Metadata 'notes' not returned from server.
└ @ FredData C:\Users\matsz\.julia\packages\FredData\5M7x4\src\get_data.jl:77
┌ Warning: Metadata 'notes' not returned from server.
└ @ FredData C:\Users\matsz\.julia\packages\FredData\5M7x4\src\get_data.jl:77


211×13 TimeArray{Float64, 2, Date, Matrix{Float64}} 1971-10-01 to 2024-04-01
┌────────────┬──────────────┬───────────────┬─────────────────┬─────────────────
│            │ empty        │ NGDPRSAXDCPLQ │ POLCPIALLQINMEI │ IRLTLT01PLM156 ⋯
├────────────┼──────────────┼───────────────┼─────────────────┼─────────────────
│ 1971-10-01 │    6.47e-322 │           NaN │             NaN │             Na ⋯
│ 1972-01-01 │   1.764e-321 │           NaN │             NaN │             Na ⋯
│ 1972-04-01 │     4.1e-322 │           NaN │             NaN │             Na ⋯
│ 1972-07-01 │ 1.08086e-311 │           NaN │             NaN │             Na ⋯
│ 1972-10-01 │    5.63e-322 │           NaN │             NaN │             Na ⋯
│ 1973-01-01 │    1.78e-321 │           NaN │             NaN │             Na ⋯
│ 1973-04-01 │   3.083e-321 │           NaN │             NaN │             Na ⋯
│ 1973-07-01 │     1.5e-322 │           NaN │             NaN │             Na ⋯
│     ⋮      │      ⋮       │   

In [171]:
using CSV
CSV.write("macro_data.csv", macro_data[:,[:NGDPRSAXDCPLQ, :POLCPIALLQINMEI, :IRLTLT01PLM156N, :IR3TIB01PLQ156N, :NGDPRSAXDCUSQ, :USACPIALLMINMEI, :IRLTLT01USQ156N, :IR3TIB01USQ156N, :CLVMEURSCAB1GQEA19, :CP0000EZ19M086NEST, :IRLTLT01EZM156N, :IR3TIB01EZQ156N]])

"macro_data.csv"